In [1]:
import numpy as np
import pandas as pd
import glob
import os
import xlrd
import openpyxl


In [31]:
# Read S&P Global indices
folder_path = 'raw'

# Get all .xls files in the folder
xls_files = glob.glob(os.path.join(folder_path, '*.xls'))

# Read all files
dataframes = []
for file in xls_files:
    df = pd.read_excel(file, skiprows=6, usecols=[0, 1]).iloc[:-1].dropna(how='all')
    df['Effective date '] = pd.to_datetime(df['Effective date '])
    dataframes.append(df)

df_merged = dataframes[0]
for idx in range(1, len(dataframes)):
    df_merged = df_merged.merge(dataframes[idx],  how='outer', on='Effective date ',left_index=False, right_index=False)

In [32]:
# Read two MSCI indices from msci file
msci = pd.read_excel('raw/msci.xlsx', skiprows=5, usecols=[0, 1, 2]).dropna(how='all')
msci['Date'] = pd.to_datetime(msci['Date'])

# Merge msci indices with S&P global indices
df_merged = df_merged.merge(msci,  how='left', left_on='Effective date ', right_on = 'Date', 
                            left_index=False, right_index=False).drop(columns='Date')

In [ ]:
# Read vix
vix = pd.read_csv('raw/vix.csv').dropna(how='all')[['Date','vix']].dropna(how='all')
vix['Date'] = pd.to_datetime(vix['Date'])
df_merged = df_merged.merge(vix,  how='left', left_on='Effective date ', right_on = 'Date', 
                            left_index=False, right_index=False).drop(columns='Date')

In [ ]:
# Rename columns
df_merged = df_merged.rename(columns={'Effective date ': 'date',
                          'S&P US Dollar Futures Index TR': 'us_dollar',
                          'S&P GSCI TR': 'commodities',
                          'S&P Composite 1500 (TR)':'us_equity',
                          'S&P 500 Value (TR)': 'value',
                          'S&P 600 (TR)': 'small_cap',
                          'S&P 500 Low Volatility Index TR':'low_vol',
                          'S&P U.S. High Yield Corporate Bond Index': 'credit',
                          'S&P U.S. Treasury Bond 10+ Year Index':'interest_rate',
                          'S&P 500 Momentum Index (US Dollar) Gross Total Return':'momentum',
                          'S&P GSCI (U.S. 10Y TIPS) (USD) TR':'inflation',
                          'S&P Bitcoin Index (USD)':'bitcoin',
                          'S&P 500 Quality (US Dollar) Gross Total Return':'quality',
                          'S&P Equity Trend (5 Region) Long/Short Index (USD) TR':'trend',
                          'MSCI World Index':'equity',
                          'MSCI EM (Emerging Markets) Index':'em_equity',
                          'vix':'long_vol'})
df_merged.set_index('date', inplace=True)

In [ ]:
# Map assets to categories
cat_map = {'core_macro': ['commodities', 'credit', 'interest_rate', 'equity'],
           'secondary_macro': ['us_dollar', 'us_equity', 'inflation', 'bitcoin', 'em_equity'],
           'macro_styles': ['trend', 'long_vol'],
           'equity_styles': ['value', 'small_cap', 'low_vol','momentum','quality']}

In [56]:
# Output dataframe and dictionary to a pickle file
import pickle

with open('data_and_categories.pkl', 'wb') as f:
    pickle.dump({'data': df_merged, 'categories': cat_map}, f)

In [57]:
# Load pickle file
with open('data_and_categories.pkl', 'rb') as f:
    saved = pickle.load(f)
    data = saved['data']
    cat_map = saved['categories']